# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [1]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
import numpy as np
import pandas as pd
from itertools import combinations

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [33]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        # Modified code by Miguel
        self.engletters = np.array([ s for s in 'abcdefghijklmnopqrstuvwxyz'])
        self.guessed_letters = []
        self.available_letters = np.array([ s for s in 'abcdefghijklmnopqrstuvwxyz'])
        self.found_patterns = {}
        ###

        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        # Modified code by Miguel
        self.data = pd.DataFrame(self.full_dictionary, columns=['words']).fillna('null')
        self.data['length'] = self.data['words'].str.len()
        self.data['word_sep'] = [list(w) for w in self.data['words']]
        self.w2 = {}
        self.w2_total = {}
        ###
        
    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess_bench(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word[::2].replace("_",".")
        
        # find length of passed word
        len_word = len(clean_word)
        
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        
        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue
                
            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word,dict_word):
                new_dictionary.append(dict_word)
        
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary
        
        
        # count occurrence of all characters in possible word matches
        full_dict_string = "".join(new_dictionary)
        
        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common()                   
        
        guess_letter = '!'
        
        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        for letter,instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                guess_letter = letter
                break
            
        # if no word matches in training dictionary, default back to ordering of full dictionary
        if guess_letter == '!':
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter,instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break            
        
        return guess_letter

    ##########################################################
    #####Miguel's implementation of guess using entropy ######
    ##########################################################
    def surprise(self,x):
        if(x!=0): return -1*x*np.log2(x)
        else: return 0

    def is_pattern(self,sep_word, let, pattern):
        pos = ''.join(['0' if aa!=let else '1' for aa in sep_word])
        return pos==pattern

    def get_pattern(self,sep_word, let):
        pos = ''.join(['0' if aa!=let else '1' for aa in sep_word])
        return pos

    def guess(self,word):
        wo = word.split(' ')[:-1]
        L = len(wo)
        if(len(self.guessed_letters)>=1):
            let = self.guessed_letters[-1]
            self.found_patterns[let] = self.get_pattern(wo,let)
            if(len(self.w2)!=0):
                self.w2 = self.w2[[self.is_pattern(aa,let,self.found_patterns[let]) for aa in self.w2['word_sep']]]
        else:
            self.w2 = self.data[self.data['length']==L]
            self.w2_total = self.w2.copy()
            self.available_letters = self.engletters
            self.found_patterns = {}
        
        if(len(self.w2)>4):
            probabilities = {}

            for i in range(L+1):
                combs = combinations(range(L),i)
                for c in list(combs):
                    b = ''.join(['1' if j in c else '0' for j in range(L)])
                    probabilities[b] = 0
            PT = []
            H = []
            cnt = 0
            for l in self.available_letters:
                PT.append(probabilities.copy())
                for a in self.w2['word_sep']:
                    pos = ''.join(['0' if aa!=l else '1' for aa in a])
                    PT[cnt][pos]+=1/len(self.w2)
                H.append(sum(self.surprise(PT[cnt][x]) for x in PT[cnt]))
                cnt += 1

            let = self.available_letters[np.argmax(H)]
            self.available_letters = np.delete(self.available_letters,np.where(self.available_letters==let)[0])
        elif(len(self.w2)==1):
            ws = list(self.w2['word_sep'])[0]
            poss = [t for t in ws if t in self.available_letters]
            let = poss[0]
            self.available_letters = np.delete(self.available_letters,np.where(self.available_letters==let)[0])
        else:
            # Divide and try to solve each part based on roots or prefixes and suffixes
            sofar1, sofar2 = wo[:L//2], wo[L//2:]
            l1 = len([i for i in range(len(sofar1)) if sofar1[i]=='_'])
            l2 = len([i for i in range(len(sofar2)) if sofar2[i]=='_'])
            coin = np.random.random()
            if(l1>l2):
                indices  = [True if(re.match('^'+"".join(sofar1).replace('_','.')+'\w+',w)) else False for w in self.data['words']]
                w2_new = self.data[indices]
                pos = [i for i in range(len(sofar1)) if sofar1[i]=='_']
                l = 0
                
            elif(l2>l1):
                indices = [True if(re.match('\w+'+"".join(sofar2).replace('_','.')+'$',w)) else False for w in self.data['words']]
                w2_new = self.data[indices]
                pos = [i for i in range(len(sofar2)) if sofar2[i]=='_']
                l = len(sofar2)
            else:
                if(coin>0.5):
                    indices  = [True if(re.match('^'+"".join(sofar1).replace('_','.')+'\w+',w)) else False for w in self.data['words']]
                    w2_new = self.data[indices]
                    pos = [i for i in range(len(sofar1)) if sofar1[i]=='_']
                    l = 0
                    #print('root')
                else:
                    indices = [True if(re.match('\w+'+"".join(sofar2).replace('_','.')+'$',w)) else False for w in self.data['words']]
                    w2_new = self.data[indices]
                    pos = [i for i in range(len(sofar2)) if sofar2[i]=='_']
                    l = len(sofar2)
                    #print('suff')

                
            if(len(w2_new)!=0):
                prob_let = dict([(a,0) for a in self.available_letters])
                for k,p in enumerate(pos):
                    for a in self.available_letters:
                        prob_let[a] += sum([aa[p-l]==a for aa in w2_new['word_sep']])/(len(w2_new))
                    
                let = self.available_letters[np.argmax([prob_let[aa] for aa in self.available_letters])]
                self.available_letters = np.delete(self.available_letters,np.where(self.available_letters==let)[0])
            
            else:
                pos = [i for i in range(len(wo)) if wo[i]=='_']
                max_prob = np.empty((len(pos),2),dtype=object)
                for k,p in enumerate(pos):
                    prob_let = {}
                    
                    for a in self.available_letters:
                        prob_let[a] = sum([aa[p]==a for aa in self.w2_total['word_sep']])/(len(self.w2_total))
                    max_prob[k] = [self.available_letters[np.argmax([prob_let[aa] for aa in self.available_letters])],
                                max([prob_let[aa] for aa in self.available_letters])]
                
                let = max_prob[np.argmax(max_prob[:,1]),0]
                self.available_letters = np.delete(self.available_letters,np.where(self.available_letters==let)[0])

        return let

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []

        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [34]:
api = HangmanAPI(access_token="7cac63f1c7b64213f6e2736e7fe7dc", timeout=2000)

## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [4]:
print(api.w2)

{}


In [5]:
api.guessed_letters,api.available_letters

([],
 array(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
        'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'],
       dtype='<U1'))

In [37]:
api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an alloted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))

Successfully start a new game! Game ID: 1f9048eb0009. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': '1f9048eb0009', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ _ _ e '}
Guessing letter: a
Sever response: {'game_id': '1f9048eb0009', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ a _ _ _ a _ e '}
Guessing letter: i
Sever response: {'game_id': '1f9048eb0009', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ i _ a _ _ _ a _ e '}
Guessing letter: s
Sever response: {'game_id': '1f9048eb0009', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ i _ a s _ _ a _ e '}
Guessing letter: t
Sever response: {'game_id': '1f9048eb0009', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ i _ a s t _ a _ e '}
Guessing letter: d
Sever response: {'game_id': '1f9048eb0009', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ i _ a s t _ a d e '}
Guessing letter: m
Sever response: {'game_id': '1f9048eb0009', 'status': 'ong

In [38]:
l = []
for i in range(100):
    l.append(api.start_game(practice=1,verbose=False))


Other exception caught on request.


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [22]:
# Never using the atomic fragmentation of the _
sum(l)/10

0.7

In [31]:
# using it only when it is more than 0.1
sum(l)/10

0.2

In [ ]:
# using it with more than 

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [ ]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    #api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [ ]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)